In [22]:
#!pip install openai

In [23]:
import requests
from bs4 import BeautifulSoup
import re
import json
 
url = "https://learn.microsoft.com/en-us/azure/ai-services/openai/tutorials/embeddings?tabs=python-new%2Ccommand-line&pivots=programming-language-python"
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
content = soup.get_text()

In [24]:
from azure.identity import DefaultAzureCredential
from openai import AzureOpenAI
 
 
 
token = DefaultAzureCredential().get_token("https://cognitiveservices.azure.com/.default").token
 
endpoint = "https://aoai-l-eastus2.openai.azure.com/"
model_name = "gpt-4.1"
deployment_name = "gpt-4.1"
api_version = "2024-12-01-preview"
 
llm = AzureOpenAI(
    api_version=api_version,
    azure_endpoint=endpoint,
    api_key=token,
)

In [25]:
import requests
from bs4 import BeautifulSoup, Comment
import re

def scrape_sections(url):
    """
    Fetch URL, parse its main content, and split into sections at the first <h1> then each <h2>.
    Returns a list of dicts:
      [
        {
          "title" : "Heading Text",
          "tag": "h1"|"h2",
          "text": "...clean plain text under heading...",
          "images": [...all <img> tags as strings...],
          "img_alt": [...alt text from images...],
          "tables": [...all <table>...</table> as strings...],
          "codes": [...all <code>...</code> as strings...]
        },
        ...
      ]
    """
    # 1) Fetch and parse the page
    resp = requests.get(url)
    resp.raise_for_status()
    soup = BeautifulSoup(resp.text, "html.parser")

    # 2) Locate main content
    main = (
        soup.find("main")
        or soup.find(id="main")
        or soup.find("content")
        or soup.body
    )
    if main is None:
        return []

    # 3) Identify the first <h1> and all following <h2> tags
    all_heads = main.find_all(["h1", "h2"])
    sections = []
    seen_h1 = False
    for tag in all_heads:
        if tag.name == "h1" and not seen_h1:
            sections.append(tag)
            seen_h1 = True
        elif tag.name == "h2" and seen_h1:
            sections.append(tag)

    # 4) Extract content under each heading
    output = []
    for idx, heading in enumerate(sections):
        end_tag = sections[idx + 1] if idx + 1 < len(sections) else None

        content_nodes = []
        for elem in heading.next_elements:
            if elem == end_tag:
                break
            if isinstance(elem, str) or getattr(elem, 'name', None):
                content_nodes.append(elem)

        section_html = "".join(str(n) for n in content_nodes)
        section_soup = BeautifulSoup(section_html, "html.parser")

        # Remove scripts, styles, and comments
        for tag in section_soup(["script", "style"]):
            tag.decompose()
        for comment in section_soup.find_all(string=lambda text: isinstance(text, Comment)):
            comment.extract()

        # Extract clean text
        plain_text = section_soup.get_text(separator=" ", strip=True)
        plain_text = re.sub(r'\s+', ' ', plain_text)
        plain_text = re.sub(r'(Feedback\s*){2,}', 'Feedback ', plain_text)
        plain_text = re.sub(r'(\d{4}-\d{2}-\d{2}\s*){2,}', r'\1', plain_text)

        # Extract other elements
        heading_text = heading.get_text(strip=True)
        img_alt = [img.get("alt", "") for img in section_soup.find_all("img")]
        images = [str(img) for img in section_soup.find_all("img")]
        tables = [str(tbl) for tbl in section_soup.find_all("table")]
        codes = [str(code) for code in section_soup.find_all("code")]

        info = {
            "title": heading_text,
            "tag": heading.name,
            "text": plain_text,
            "images": images,
            "img_alt": img_alt,
            "tables": tables,
            "codes": codes
        }
        output.append(info)

    return output


In [26]:
sections = scrape_sections("https://learn.microsoft.com/en-us/azure/load-balancer/gateway-overview")
# for sec in sections:
#     for title, data in sec.items():
#         print(title, data["tag"], "→", data["text"][:80], "…")
#         print("Images:", data["images"])
#         print("Tables:", data["tables"])
#         print("Codes:", data["codes"])
#         print("Image Alts:", data["img_alt"])
#         print()


In [27]:
import re

paragraph = sections[1]["text"]
second_sentence_start = sections[2]["text"][:20]

# Escape special regex characters
escaped_sentence_start = re.escape(second_sentence_start)

pattern = fr"^(.*?)(?={escaped_sentence_start}.*?\.)"

match = re.search(pattern, paragraph)

if match:
    sections[1]["text"] = match.group(1)
else:
    print("No match found.")


In [28]:
sections

[{'title': 'Gateway Load Balancer',
  'tag': 'h1',
  'text': 'Gateway Load Balancer 2024-08-26 Feedback 2024-08-26 Feedback In this article',
  'images': [],
  'img_alt': [],
  'tables': [],
  'codes': []},
 {'title': 'In this article',
  'tag': 'h2',
  'text': "In this article Gateway Load Balancer is a SKU of the Azure Load Balancer portfolio catered for high performance and high availability scenarios with third-party Network Virtual Appliances (NVAs). With the capabilities of Gateway Load Balancer, you can easily deploy, scale, and manage NVAs. Chaining a Gateway Load Balancer to your public endpoint only requires one selection. You can insert appliances transparently for different kinds of scenarios such as: Firewalls Advanced packet analytics Intrusion detection and prevention systems Traffic mirroring DDoS protection Custom appliances With Gateway Load Balancer, you can easily add or remove advanced network functionality without extra management overhead. It provides the bump-in

In [7]:
#First determine the field/topic and difficult terms

# Then pass into the summarization llm. If intermediate, then assume user is professional in the field
# or topic, but don't know about this particualr project. If begginer, then unfamilier with field
# in general. For both cases, we tell it to determine if the user's background would allow them to know
# each difficult term, and define it if they wouldn't know. Of course have it summarize everything too

In [8]:
#Then pass all of that into the judge along with the original chunk and the entire page's content 
# (from all chunks). For each of the following, have a single iteration to check if it fulfills that 
# requirement. If not, refine. Then return after all iterations

#Requirements: No hallucinations, didn't leave any necessary information out. No irrelevent info. Did it lose the meaning


In [9]:
full_webpage_text = "\n\n".join([
        f"{section['title']}\n{section['text']}"
        for section in sections
    ])

In [10]:
difficulty_detection_prompt = """You are a 'difficultly detection' LLm. You are a component of a larger process designed to simplifiy texts. 

## Context
### The first LLM of this process is to identify the topic as well as difficult terms and the purpose of the document, the next LLM revises the text based on the user's expertise in the field to make it more comprehensible and easier to read, and the last LLM iteratively improves on the summarization to ensure certain quality requirements are met. 

**You are the first step in this process, and so your job is to identify the topic and the difficult terms and the overall purpose of the document.**

## Instructions
 - You will be provided two bits of information: The full webpage text where the chunk comes from, and the specific text chunk that will later be summarized.

 - You may also be given the alternate text for images or tables if those were present in the section chunk. If those are empty, assume there were no images or tables.

 - **Generate the difficult terms from the text chink. If the tables or alternate image text are available, consider them as part of the text chunk, and use them for listing the difficult terms as well**

 - **Use the full webpage text to detemine the topic. DO NOT USE IT FOR THE DIFFICULT TERMS LIST. Also use the full webpage text to determine the purpose of the text.**

 - Format your response as a valid JSON string, where strings are wrapped in double quotes, using backslash escapes.

**Structure your response according to the following template: {{"topic" : "__topic__", "purpose_of_document" : "__purpose__", "difficult_terms" : []}}**

The topic and purpose must be returned as strings, and the difficult terms must be a list of just the terms. Do not include any other information.

"""

In [11]:
for section in sections:
    specific_text = section["text"]
    tables = section["tables"]
    image_alt_text = section["img_alt"]

    response = llm.chat.completions.create(
    model= deployment_name,
    messages=[
        {
        "role":"system",
        "content": difficulty_detection_prompt
        },
        {
        "role":"user",
        "content": f"## Full Webpage Text\n{full_webpage_text}\n## Specific Text\n{specific_text}\n## Tables\n{tables}\n## Image Alt Texts\n{image_alt_text}\n"
        }
    ],
    max_tokens=4096
    )

    response_content = response.choices[0].message.content

    pattern = r'\{(?:[^{}]|(?:\\{[^{}]*\\}))*\}'
    cleaned_response = re.findall(pattern, response_content)[0]

    cleaned_response_json = json.loads(cleaned_response)

    section["topic_difficult_terms"] = cleaned_response_json


In [12]:
def intermediate_rewrite(sections):
    
    token = DefaultAzureCredential().get_token("https://cognitiveservices.azure.com/.default").token
    
    endpoint = "https://aoai-l-eastus2.openai.azure.com/"
    model_name = "gpt-4.1"
    deployment_name = "gpt-4.1"
    api_version = "2024-12-01-preview"
    
    llm = AzureOpenAI(
        api_version=api_version,
        azure_endpoint=endpoint,
        api_key=token,
    )


    system_prompt = """
    You are a content revision and teaching assistant. You are a component of a larger process designed to simplify complicated documentations.

    ## Context
    You are given a section of a technical documentation and the content of the whole page for context. Your task is to simplify the content of that section only, making it easier to understand for a user who is a professional or student in the field, but not familiar with this particular topic.

    ## Instructions
    - **You need to re-write the content given in 'text' only**
    - You have access to the section title ('title'), text content ('text'), image alt ('image_alt') texts of the images within the section, the main topic of the page ('topic'), the purpose of the document (purpose_of_document), the difficult terms in the document (difficult_terms).
    - Use simple language and short sentences.
    - Do not add any new information and do not omit any existing information, except the CODE. **OMIT ALL CODE** \
    - Analyse the purpose of the document in "purpose_of_document" to decide if you need to explain the difficult terms given in "difficult_terms" appearing within that section, for example if it is an instructional document, don't explain the difficult terms; if it's a tutorial, explain a bit depending on the professional/student's potential knowledge.
    - Assume that after the revised section content, the ORIGINAL code, then the images, then the original tables will be displayed, in that order. **Integrate ONLY the references and explanations to them in the revised content, RETURN ONLY PLAIN TEXT CONTENT.**
    - If 'text' is empty, return empty string.

    - **Return just the revised section content AS PLAIN TEXT, without any additional text or explanations.**
    """

    for section in sections:
        # 1) Build a structured payload
        payload = {
            "title": section["title"],
            "text": section['text'],
            "img_alt": section["img_alt"],
            "topic": section["topic_difficult_terms"]["topic"],
            "purpose_of_document": section["topic_difficult_terms"]["purpose_of_document"],
            "difficult_terms": section["topic_difficult_terms"]["difficult_terms"],
        }

        # 2) Serialize to JSON
        user_content = json.dumps(payload)

        # 3) Send to the LLM
        response = llm.chat.completions.create(
            model=deployment_name,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user",   "content": user_content}
            ],
            max_tokens=4096,
            temperature=1.0,
            top_p=1.0,
        )


        section['intermediate_level_text'] = response.choices[0].message.content

In [13]:
intermediate_rewrite(sections)

In [14]:
def beginner_rewrite(sections):
    
    token = DefaultAzureCredential().get_token("https://cognitiveservices.azure.com/.default").token
    
    endpoint = "https://aoai-l-eastus2.openai.azure.com/"
    model_name = "gpt-4.1"
    deployment_name = "gpt-4.1"
    api_version = "2024-12-01-preview"
    
    llm = AzureOpenAI(
        api_version=api_version,
        azure_endpoint=endpoint,
        api_key=token,
    )


    system_prompt = """
    You are a content revision and teaching assistant. You are a component of a larger process designed to simplify complicated documentations.

    ## Context
    You are given a section of a technical documentation and the content of the whole page for context. Your task is to simplify the content of that section only, making it so simple and easy to understand that even a user who is **Zero knowledge** of the field, **No familiarity** with the topic.

    ## Instructions
    - **You need to re-write the content given in 'text' only**
    - You have access to the section title ('title'), text content ('text'), image alt ('image_alt') texts of the images within the section, the main topic of the page ('topic'), the purpose of the document ('purpose_of_document'), the difficult terms in the document ('difficult_terms').
    - Use extremely simple language, examples and short sentences, fit for anyone with basic education to understand.
    - Do not add any new information and do not omit any existing information, except the CODE. **OMIT ALL CODE FROM TEXT** \
    - Analyse the purpose of the document in "purpose_of_document" and explain the difficult terms given in "difficult_terms" appearing within that section keeping the purpose in mind.
    - Assume that after the revised section content, the ORIGINAL code, then the images, then the original tables will be displayed, in that order. **Integrate ONLY the references and explanations to them in the revised content, RETURN ONLY PLAIN TEXT CONTENT.**
    - If 'text' is empty, return empty string.

    - **Return just the revised section content AS PLAIN TEXT, without any additional text or explanations.**
    """

    for section in sections:
        # 1) Build a structured payload
        payload = {
            "title": section["title"],
            "text": section['text'],
            "img_alt": section["img_alt"],
            "topic": section["topic_difficult_terms"]["topic"],
            "purpose_of_document": section["topic_difficult_terms"]["purpose_of_document"],
            "difficult_terms": section["topic_difficult_terms"]["difficult_terms"],
        }

        # 2) Serialize to JSON
        user_content = json.dumps(payload)

        # 3) Send to the LLM
        response = llm.chat.completions.create(
            model=deployment_name,
            messages=[
                {"role": "system", "content": system_prompt},
                {"role": "user",   "content": user_content}
            ],
            max_tokens=4096,
            temperature=1.0,
            top_p=1.0,
        )


        section['beginner_level_text'] = response.choices[0].message.content

In [15]:
beginner_rewrite(sections)

In [16]:
sections

[{'title': 'Gateway Load Balancer',
  'tag': 'h1',
  'text': 'Gateway Load Balancer 2024-08-26 Feedback 2024-08-26 Feedback In this article',
  'images': [],
  'img_alt': [],
  'tables': [],
  'codes': [],
  'topic_difficult_terms': {'topic': 'Azure Gateway Load Balancer',
   'purpose_of_document': 'To explain the features, components, configuration, scenarios, limitations, and use cases of the Azure Gateway Load Balancer for integrating Network Virtual Appliances and managing network traffic.',
   'difficult_terms': []},
  'intermediate_level_text': 'Gateway Load Balancer\n\nThis article is about the Azure Gateway Load Balancer.',
  'beginner_level_text': 'Gateway Load Balancer\n\nThis article talks about Gateway Load Balancer. It includes information about what it is, what it can do, its parts, how to set it up, where it can be used, and some things it cannot do. It also gives real-life examples of how you can use it. This tool helps connect other network devices and manage how data 

In [17]:
consitution = ["Does not hallucinate information and specifically does not add incorrect information. The entire response should be accurate.",
                "Does not leave any unecessary information out. All important information from the original text is present in the summarization.",
                "Does not include irrelevent information. Information whose inclusion would confuse or dilute the summary is not present.",
                "The overall meaning of the text is not lost. No information that detracts or would lose the understanding of the original text is present in the summary."]

In [18]:
judge_prompt = """You are a 'simplified text revision judge' LLm. You are a component of a larger process designed to simplifiy texts. 

## Context
### The first LLM of this process is to identify the topic as well as difficult terms and the purpose of the document, the next LLM revises the text based on the user's expertise in the field to make it more comprehensible and easier to read, and the last LLM iteratively improves on the summarization to ensure certain quality requirements are met. 

**You are the third and last step in this process, and so your job is to iterate on the current simplification to improve it.**

## Instructions
 - You will be provided three bits of information: The complete webpage the info comes from, and the specific text that was simplified, and the simplified text itself.

 - You will be given a specific "requirement" of the simplified text and your job is to check if the simplified text fulfills the requirement. This may require you to look at all 3 pieces of information.

 - **Make sure to check if the simplified text fulfills the requirement**. 
 
 - **If the simplified text does not fulfill the requirement, revise the simplified text to improve it and fulfill the requirement".**

 - Format your response as a valid JSON string, where strings are wrapped in double quotes, using backslash escapes.

**Structure your response according to the following template: {{"new_simplified_text" : "__simplified_text__"}}**

**If the simplified text is correct, so it does indeed fulfill the requirement, then return the simplified text as is. Still label it as "new_simplified_text" in the json.**


There is no need to give any additional information or reason for revision. Give just the new revision.
"""

In [20]:
for section in sections:
    specific_text = section["text"]
    tables = section["tables"]
    image_alt_text = section["img_alt"]
    simplified_intermediate_text = section["intermediate_level_text"]
    simplified_begginer_text = section["beginner_level_text"]

    for rule in consitution:
        response = llm.chat.completions.create(
        model= deployment_name,
        messages=[
            {
            "role":"system",
            "content": judge_prompt
            },
            {
            "role":"user",
            "content": f"## Full Webpage Text\n{full_webpage_text}\n## Specific Text\n{specific_text}\n## Simplified Text\n{simplified_intermediate_text}\n## Rule\n{rule}\n"
            }
        ],
        max_tokens=4096
        )

        response_content = response.choices[0].message.content

        pattern = r'\{(?:[^{}]|(?:\\{[^{}]*\\}))*\}'
        cleaned_response = re.findall(pattern, response_content)[0]

        cleaned_response_json = json.loads(cleaned_response)

        section["intermediate_level_text"] = cleaned_response_json





        response = llm.chat.completions.create(
        model= deployment_name,
        messages=[
            {
            "role":"system",
            "content": judge_prompt
            },
            {
            "role":"user",
            "content": f"## Full Webpage Text\n{full_webpage_text}\n## Specific Text\n{specific_text}\n## Simplified Text\n{simplified_begginer_text}\n"
            }
        ],
        max_tokens=4096
        )

        response_content = response.choices[0].message.content

        pattern = r'\{(?:[^{}]|(?:\\{[^{}]*\\}))*\}'
        cleaned_response = re.findall(pattern, response_content)[0]

        cleaned_response_json = json.loads(cleaned_response)

        section["beginner_level_text"] = cleaned_response_json


In [21]:
sections

[{'title': 'Gateway Load Balancer',
  'tag': 'h1',
  'text': 'Gateway Load Balancer 2024-08-26 Feedback 2024-08-26 Feedback In this article',
  'images': [],
  'img_alt': [],
  'tables': [],
  'codes': [],
  'topic_difficult_terms': {'topic': 'Azure Gateway Load Balancer',
   'purpose_of_document': 'To explain the features, components, configuration, scenarios, limitations, and use cases of the Azure Gateway Load Balancer for integrating Network Virtual Appliances and managing network traffic.',
   'difficult_terms': []},
  'intermediate_level_text': {'new_simplified_text': 'Gateway Load Balancer\n\nThis article introduces the Azure Gateway Load Balancer.'},
  'beginner_level_text': {'new_simplified_text': 'Gateway Load Balancer\n\nThis article is about Gateway Load Balancer. It explains what Gateway Load Balancer is, how it works, its main features, components, how to set it up, where you can use it, its limitations, and examples of its use. Gateway Load Balancer helps you connect and

In [ ]:
kyle_format_easy = []

for i, section in enumerate(sections):
    
    temp = {}
    temp["id"] = f"section-{i}"
    temp["title"] = section["title"]
    
    content = []
    content.append({"type" : "paragraph", "text" : [section["beginner_level_text"]]})

    temp["content"] = content

    kyle_format_easy.append(temp)

In [ ]:
kyle_format_easy

In [ ]:
kyle_format_intermediate = []

for i, section in enumerate(sections):
    
    temp = {}
    temp["id"] = f"section-{i}"
    temp["title"] = section["title"]
    
    content = []
    content.append({"type" : "paragraph", "text" : [section["beginner_level_text"]]})

    temp["content"] = content

    kyle_format_intermediate.append(temp)

In [ ]:
kyle_format_intermediate